In [1]:
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table
from astromodels import Powerlaw,  PointSource, SpectralComponent
import astropy.time as at
from datetime import datetime
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
import os
from MultinestClusterFit import powerlaw_binned_spectrum
import pickle

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available


/home/moej56153/.pyenv/versions/MT/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)


/home/moej56153/.pyenv/versions/MT/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


[WARNING ] ROOT minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] No fermitools installed
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML
[WARNING ] Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML
[WARNING ] Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML
Using the irfs that are valid between Start and 03/07/06 06:00:00 (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 03/07/06 06:00:00 and 04/07/17 08:20:06 (YY/MM/DD HH:MM:SS)
Using the ir

In [33]:
data_path = "crab_data/0374"

In [34]:
source_ra, source_dec = 10, -40
source_piv = 100.
source_Ks = [0.5e-4, 2e-4, 8e-4]
source_indices = [-0.5, -2, -8]

In [35]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_Ks, source_indices), f)


In [36]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [37]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [38]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [39]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [40]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [41]:
for i in range(len(source_Ks)):
    for j in range(len(source_indices)):
        
        temp_path = f"{data_path_d}/{i}_{j}"        
        
        if not os.path.exists(temp_path):
            os.mkdir(temp_path)
            
        os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
        os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

                
        hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
        hdu.writeto(f"{temp_path}/dead_time.fits")
        
        source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_Ks[i], source_indices[j])
        
        total_counts = background_counts.copy()
        total_counts["COUNTS"] += source_counts
                
        hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
        hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [ ]:
# second source

In [2]:
data_path = "crab_data/0374"
# data_path = "crab_data/1380"

In [3]:
# primary

source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

# secondary
s_source_decs = [-40, -40.5, -41.5, -45, -65, -85]
s_source_Ks = [0.1e-4, 0.3e-4, 1e-4]
s_source_index = -3

In [4]:
pointing_index = 1
# pointing_index = 4


data_path_d = "simulated_data/0374_const_bkg_sec_source_i_3"
# data_path_d = "simulated_data/1380_const_bkg_sec_source"


if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, s_source_decs, s_source_Ks, s_source_index), f)


In [5]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [6]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [7]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [8]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [12]:
s_resp_mats = []

for i in range(len(s_source_decs)):
    t_resp_mats = []
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        time = time_start[p_i]
        dets = get_live_dets(time=time, event_types=["single"])
        
        rmfs = []
        for d in dets:
            rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
            
        sds = np.empty(0)
        for d in range(len(dets)):
            sd = SPIDRM(rmfs[d], source_ra, s_source_decs[i])
            sds = np.append(sds, sd.matrix.T)
        t_resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))
    s_resp_mats.append(t_resp_mats)

In [13]:
def calc_count_rates_sec(resp_mats, ra, dec, piv, K, index, s_resp_mats, s_dec, s_K, s_index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    s_pl = Powerlaw()
    s_pl.piv = piv
    s_pl.K = s_K
    s_pl.index = s_index
    component1 = SpectralComponent("pl", shape=s_pl)
    s_source = PointSource("Test", ra=ra, dec=s_dec, components=[component1])
    
    s_spec = s_source(emod)
    s_spec_binned = powerlaw_binned_spectrum(emod, s_spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1))
    
    s_source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1))
        
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
                
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
                        
        s_resp_mat = s_resp_mats[p_i]
        
        s_count_rates = np.dot(s_spec_binned, s_resp_mat)
                    
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            s_source_counts[index,:] = np.random.poisson(s_count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
            
    total_source_counts = source_counts + s_source_counts
    
    return total_source_counts

In [18]:
for i in range(len(s_source_decs)):
    for j in range(len(s_source_Ks)):
        
        temp_path = f"{data_path_d}/{i}_{j}"        
        
        if not os.path.exists(temp_path):
            os.mkdir(temp_path)
            
        os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
        os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

                
        hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
        hdu.writeto(f"{temp_path}/dead_time.fits")
        
        source_counts = calc_count_rates_sec(resp_mats, source_ra, source_dec, source_piv, source_K, source_index, s_resp_mats[i], s_source_decs[i], s_source_Ks[j], s_source_index)
        
        total_counts = background_counts.copy()
        total_counts["COUNTS"] = source_counts + total_counts["COUNTS"]
                
        hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
        hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# energy ranges

data_path = "crab_data/0374"

In [4]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

energy_ranges = [(None, 80),
                 (None, 200),
                 (None, 1000),
                 (None, None),
                 (80, None),
                 (200, None),
                 (1000, None)]

In [5]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_energy_ranges"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, energy_ranges), f)


In [6]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [7]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [8]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [9]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [9]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [10]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# identical repeats

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

In [5]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_identical"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index), f)


In [6]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [7]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [8]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [9]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [10]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [11]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# number of energy bins

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

bin_number = [5, 25, 125, 600]

In [4]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_number_of_e_bins"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, bin_number), f)


In [5]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [6]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [7]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [8]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [ ]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [ ]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# length of SCWs

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

time_fraction = [1, 0.3, 0.1]

In [4]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_scaled_time_or_pairs"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, time_fraction), f)


In [6]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [7]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [8]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])

b_cs = []

for frac in range(len(time_fraction)):

    background_counts = t.copy()

    for i in range(int(len(background_counts) / 85)):
        if i == pointing_index:
            continue
        else:
            background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
            
    background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"] * time_fraction[frac])
    b_cs.append(background_counts)

In [9]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [27]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index, time_elapsed):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [28]:
for i in range(len(time_fraction)):
        
    temp_path = f"{data_path_d}/t_{i}"        

    if not os.path.exists(temp_path):
        os.mkdir(temp_path)
        
    os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
    os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

    temp_time = updated_time.copy()
    temp_time["LIVETIME"] *= time_fraction[i]
    hdu = fits.BinTableHDU(data=temp_time, name="SPI.-OBS.-DTI") # is all of this correct?
    hdu.writeto(f"{temp_path}/dead_time.fits")

    source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index, time_elapsed*time_fraction[i])

    total_counts = b_cs[i].copy()
    total_counts["COUNTS"] += source_counts
            
    hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
    hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# cluster sizes

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

cluster_sizes = [2, 3, 4]

In [4]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_cluster_sizes"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, cluster_sizes), f)


In [13]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [14]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [15]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [16]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [ ]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [ ]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [ ]:
# const const bkg

In [2]:
data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 100.
source_K = 8e-4
source_index = -0.5

In [4]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_const_bkg_20"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index), f)


In [5]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [6]:
updated_time

DEADRATIO,LIVETIME
,s
float32,float64
0.8488644,2285.7130358229892
0.8492773,2286.8248684576347
1.0,0.0
0.848927,2285.8817672839295
0.84870106,2285.2732544857345
0.8490433,2286.1948843688656
0.84908646,2286.3110680690056
0.8498843,2288.4593072045127


In [7]:
time_elapsed

array([ 501.36984114,  501.6546314 ,    0.        , ...,    0.        ,
       2382.83614227,    0.        ])

In [8]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [9]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [10]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)



WARNING NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.



In [11]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [15]:
time_elapsed

array([ 501.36984114,  501.6546314 ,    0.        , ...,    0.        ,
       2382.83614227,    0.        ])

In [13]:
updated_time

DEADRATIO,LIVETIME
,s
float32,float64
0.8488644,2285.7130358229892
0.8492773,2286.8248684576347
1.0,0.0
0.848927,2285.8817672839295
0.84870106,2285.2732544857345
0.8490433,2286.1948843688656
0.84908646,2286.3110680690056
0.8498843,2288.4593072045127


In [ ]:
for i in range(len(source_Ks)):
    for j in range(len(source_indices)):
        
        temp_path = f"{data_path_d}/{i}_{j}"        
        
        if not os.path.exists(temp_path):
            os.mkdir(temp_path)
            
        os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
        os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

                
        hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
        hdu.writeto(f"{temp_path}/dead_time.fits")
        
        source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_Ks[i], source_indices[j])
        
        total_counts = background_counts.copy()
        total_counts["COUNTS"] += source_counts
                
        hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
        hdu.writeto(f"{temp_path}/evts_det_spec.fits")